In [31]:
import re
import nltk
import codecs
import networkx as nx
from bs4 import BeautifulSoup

In [2]:
G = nx.read_gexf("influences.gexf")
names = G.nodes()

In [15]:
stopwords = codecs.open("stopwords.final.txt", "r", "utf-8").read().split()

In [50]:
def save(name, sentences):
  codecs.open("sentences/%s.txt" % name.replace("/","-"), "w", "utf-8").write("\n".join(sentences))

def load(name):
  txt = codecs.open("pages/%s.html" % name.replace("/", "-"),"r", "utf-8-sig").read()

  txt = BeautifulSoup(txt, "html.parser")
  txt = txt.get_text()
  
  return txt

def extract_sentences(name):
  sentences = []
  
  txt = load(name)
  
  txt = txt.replace(",", "")
  txt = txt.replace(";", "")
  txt = txt.replace(":", "")
  txt = txt.replace("\"", "")
  txt = txt.replace("(", "")
  txt = txt.replace(")", "")
  txt = txt.replace("[edit]", "")
  txt = txt.replace(u"•", "")

  txt = re.sub(r'\[[0-9]+\]', '', txt)
  txt = re.sub(r'ISBN [0-9\-]+', '', txt)
  txt = re.sub(r'(?P<a>[a-z])(?P<b>[A-Z])', '\g<a> \g<b>', txt)
  txt = re.sub(r'(?P<a>[0-9])(?P<b>[A-Za-z])', '\g<a> \g<b>', txt)
  
  sentences = txt.split(".")
  
  return sentences

def clean_sentences(sentences):
  sentences = [ s.strip() for s in sentences ]

  sentences = map(filter_stopwords, sentences)

  # keep only sentences with at least 2 words
  sentences = [ s for s in sentences if len(s.split()) > 1 ]
  
  return sentences

def filter_stopwords(sentence):
  sentence = [ w.lower() for w in sentence.split() if w.lower() not in stopwords ]
  
  sentence = [ w for w in sentence if w != "" ]
  
  return " ".join(sentence)

for name in names[0:50]:
  sentences = extract_sentences(name)
  sentences = clean_sentences(sentences)

  save(name, sentences)

In [51]:
for name in names:
  sentences = extract_sentences(name)
  sentences = clean_sentences(sentences)

  save(name, sentences)